# Simulation methods: basic laws and inverse method

We here implement functions that generate samples following a specific law, from the random number generator.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial
%matplotlib inline

## Bernoulli simulation

We here want to simulate a sample that follow the Bernoulli distribution, using the inverse transform method, then show that the central limit theorem holds with this distrribution.

In [ ]:
def bernoulli(n, p=0.3):
    return np.array([int(np.random.random() < p) for _ in range(n)])

gaussian = lambda x : 1 / np.sqrt(2 * np.pi) * np.exp(-x**2 / 2)

print("Computing the mean of a Bernoulli distribution")
B = [bernoulli(k, 0.3) for k in [100, 1000, 10000]]
for l in B:
    s = 0
    for x in l:
        if x == 1:
            s += 1
    print(f"List of size {len(l)}: {s/len(l)}")

Now, we are going to verity the central limit theorem with the Bernoulli distribution.
The following function is designed to be run with all distributions that have a mean and a variance.

In [ ]:
def tcl(N, f, args, mean = 0, sigma = 1):
    """Makes a sample that follows the law of the central limit.

    Args:
        N (int): number of samples
        f (function): function that makes the sample
        args (tuple): arguments to f
        mean (int, optional): theoretical mean of the law. Defaults to 0.
        sigma (int, optional): theoretical standard deviation of the law. Defaults to 1.

    Returns:
        numpy.nparray: sample that follows a normal law.
    """
    L = np.zeros(N)
    for i in range(N):
        echant = f(*args)
        mu = np.mean(echant)
        L[i] = np.sqrt(len(echant)) * (mu - mean) / sigma
    return L

Here we apply this function to the Bernoulli law.

In [ ]:
n_size = [10, 30, 40]

echant10000 = tcl(10000, bernoulli, (100, 0.3), 0.3, 0.3)
plt.hist(echant10000, density=True,ec="white")
plt.plot(np.linspace(min(echant10000), max(echant10000), 100), gaussian(np.linspace(min(echant10000), max(echant10000), 100)))
plt.axis(xmin = min(echant10000), xmax = max(echant10000), ymin = 0)
plt.show()

We here notice that the output follows a normal law $\mathcal{N}(0,1)$.

## Binomial law

Now, we are going to do the same with the binomial law. First, we need to define how to build binomial random variables.

In [ ]:
def binomial(n,N,p):
    """Generates a binomail sample of size n

    Args:
        n (int): Size of the sample
        N (int): Number of trials
        p (float): Probability of success

    Returns:
        np.ndarray: sample
    """
    return np.array(sum([bernoulli(n,p) for _ in range(N)]))

Here, we compare the disbribution of the binomial law with the empiric one.

In [ ]:
echant = np.sum(np.array([bernoulli(30,0.1) for _ in range(10000)]), axis = 1)
plt.hist(echant, density = True, ec = 'white')
X=[i for i in range(30)]
Y=[sum([factorial(30) / (factorial(i)*factorial(30-i)) * (0.1)**i * (0.9)**(30-i)]) for i in range(30)] # Theoretical distribution
plt.scatter(X,Y, color = 'red')
plt.plot()
plt.show()

Now, we proceed with verifying the central limit theorem, with a law of parameter $p=0.1$.

In [ ]:
echant10000 = tcl(10000, binomial, args = (40, 30, 0.1), mean = 30*0.1, sigma = np.sqrt(30*0.1*0.9))
plt.hist(echant10000, density=True,ec="white")
plt.plot(np.linspace(min(echant10000), max(echant10000), 100), gaussian(np.linspace(min(echant10000), max(echant10000), 100)))
plt.axis(xmin = min(echant10000), xmax = max(echant10000), ymin = 0)
plt.show()

## Unspecified law

We here want to simulate a sample that follow this law:
- $\mathbb P(X=a_1)=0.25$.
- $\mathbb P(X=a_2)=0.125$.
- $\mathbb P(X=a_3)=0.125$.
- $\mathbb P(X=a_4)=0.5$.

with $a_1=0.5$, $a_2=1$, $a_3=1.5$ and $a_4=2$.

In [ ]:
def density():
    k = np.random.random()
    if k <= 1/4:
        return 0.5
    elif 1/4 <= k <= 3/8:
        return 1
    elif 3/8 <= k <= 1/2:
        return 1.5
    else:
        return 2

Using this function, we can now generate a sample of size 1,000 of this law.

In [ ]:
echant = np.array([density() for _ in range(10000)])
plt.hist(echant, ec = 'white', bins = 4)
plt.scatter([0.5, 1, 1.5, 2], [10**4*1/4,10**4*1/8,10**4*1/8,10**4*1/2], color = 'red')
plt.show()

## Subgroups

We now want to use probabilities to split a group of $n$ people on $p$ groups randomly.

In [ ]:
def sous_groupes(n,p):
    if n % p ==0:
        L=[]
        U=[_ for _ in range(n)]
        for i in range(p):
            tmp=[]
            while len(tmp) < n/p:
                x = int(np.random.random()*len(U))
                tmp.append(U[x])
                U.pop(x)
            L.append(tmp)
        return L
    else:
        L=[]
        U=[_ for _ in range(n)]
        for i in range(p):
            tmp=[]
            while len(tmp) < n/p - 1:
                x = int(np.random.random()*len(U))
                tmp.append(U[x])
                U.pop(x)
            L.append(tmp)
        L[-1].extend(U)
        return L
    
A=sous_groupes(9,3)
print(f"We have 9 people and 3 groups:\n{A}.")
print(f"The number of people in each group is {[len(A[i]) for i in range(3)]}.")
B=sous_groupes(101,10)
print(f"We have 101 people and 10 groups:\n{B}")
print(f"The number of people in each group is {[len(B[i]) for i in range(10)]}.")


## Poisson, Weibull and exponential laws

Here are some functions that generate samples following the Poisson, Weibull and exponential laws.

In [ ]:
def poisson(lam, n):
    L=[]
    for i in range(n):
        X = 0
        c=np.exp(-lam)
        u = np.random.random()
        fct = [1]
        while u >= c:
            X += 1
            fct.append(fct[-1] * X)
            c += np.exp(-lam) * (lam ** X) / fct[-1]
        L.append(X)
    return L

def exponentielle(lam, n):
    L=[]
    for i in range(n):
        u = np.random.random()
        L.append(-np.log(1-u)/lam)
    return L

def weibull(lam, k, n):
    L=[]
    for i in range(n):
        u = np.random.random()
        L.append((-(np.log(1-u))**(1/k))/lam)
    return L

E=exponentielle(1, 100)
print(f"Exponential law: {E}")
plt.hist(E, density=True,ec="white")
plt.show()
P=poisson(1, 100)
print(f"Poisson law: {P}")
plt.hist(P, density=True,ec="white")
plt.show()
W=weibull(1, 1, 100)
print(f"Weibull law: {W}")
plt.hist(W, density=True,ec="white")
plt.show()